In [ ]:
import json
import pandas as pd
import requests
import geopy
from geopy import distance
from geopy.distance import geodesic

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

FourSquare doesn't have pagination OR offset, which complicates doing calls to compare to Yelp. I was able to get ALL the results
    from Yelp, so if I set the radius too large here, I won't get all the results in the radius because they limit their pulls to 50.

I think 1000m is too large in Helsinki, Finland: my city of choice.

In [ ]:
# api call for each station
for i in range(0,122):
    # prepare the lat and long
    x = bike_yelp_small["latitude"][i]
    lat = str(x)
    y = bike_yelp_small["longitude"][i]
    long = str(y)

    url = "https://api.foursquare.com/v3/places/search"

    params = {
        "ll": f"{lat},{long}",
        "radius":"500",
        "categories":"13065,16000,10000",
        "sort":"DISTANCE",
        "limit":"50"
    }

    headers = {
        "Accept": "application/json",
        "Authorization": <API_KEY>
    }

    response = requests.request("GET", url, params=params, headers=headers)

    data = response.text
    parse_json = json.loads(data)
    molecule = (parse_json['results'])
    
    # save response
    four_temp = pd.DataFrame(molecule)
    # Appending the temporary DataFrame to the main DataFrame
    four_df = pd.concat([four_df, four_temp], ignore_index=True)

This took a LOT of trouble shooting. I can't explain the trouble shooting that occurred. I have never written anything like this ever,
so when I finally got this to run I was absolutely ECSTATIC. The above is the finished franken-code that finally worked. I'm positive it 
is inefficient, and UGLY. BUT it WORKS!! 'Molecule' as a name above is a relic from when I was naming files based on assuming it wouldn't 
work again, and I left it there to remind me that it will work eventually if you keep changing it.

Check for duplicates in the finished data. The original database four_df has 6032 rows, and fallback_four (duplicates removed), has 6032. 

In [ ]:
fallback_four = four_df[['categories','distance','geocodes','name']]
fallback_four.drop_duplicates('name')
fallback_four

Save this data into csv, it was hard won!

In [ ]:
four_df.to_csv(r'C:\Users\ca0pa\Desktop\LHL\Python\four_df_5500rows.csv', index=False)

## Now, we need to dig a bit further into the data and pull out the lat/long data, then rejoin the dataframes

In [ ]:
four_rest_coord = pd.DataFrame(pd.json_normalize(fallback_four['geocodes']))
four_rest_coo = four_rest_coord[['main.latitude','main.longitude']]
four_rest_coo.head(1)

Drop any extra columns that you do not need

In [ ]:
four_wlatlong = pd.concat([fallback_four, four_rest_coo], axis=1, join="inner")
four_wlatlong.drop(columns='categories',inplace=True)
four_wlatlong.drop(columns='geocodes',inplace=True)
four_wlatlong

In [ ]:
### Finally, create that mergevar that will allow you to combine this data in all the crazy ways.

In [ ]:
four_wlatlong['mergevar'] = 1

## Code troubleshooting below:

In [ ]:
x = bike_yelp_small["latitude"][1]
lat = str(x)
lat

In [ ]:
bike_yelp_small["latitude"][2]

In [ ]:
y = bike_yelp_small["longitude"][i]
long = str(y)
long

In [ ]:
url = "https://api.foursquare.com/v3/places/search?ll%3D"+lat+"%2C"+long+"&radius=500&categories=13065%2C16000%2C10000&sort=DISTANCE&limit=50"
url

In [ ]:
x = bike_yelp_small["latitude"][15]
lat = str(x)
y = bike_yelp_small["longitude"][15]
long = str(y)
url = "https://api.foursquare.com/v3/places/search?ll%3D"+lat+"%2C"+long+"&radius=500&categories=13065%2C16000%2C10000&sort=DISTANCE&limit=50"
headers = {
    "Accept": "application/json",
    "Authorization": <API_KEY>
}
data = response.text
parse_json = json.loads(data)
molecule = (parse_json['results'])
    
    # save response
four_temp = pd.DataFrame(molecule)
    # Appending the temporary DataFrame to the main DataFrame
four_df = pd.concat([four_df, four_temp], ignore_index=True)

In [ ]:
range(len(bike_yelp_small))

In [ ]:
four_four = pd.concat([four_temp, four_df], ignore_index=True)
four_four.head(1)

In [ ]:
# so, 'data' is txt 'molecule' is list and 'parse_json' is dict
# pd.read_json DOES NOT WORK
data
parse_json = json.loads(data)
type(parse_json)

In [ ]:
# for i in range(0,10):
    # prepare the lat and long
    x = bike_yelp_small["latitude"][15]
    lat = str(x)
    y = bike_yelp_small["longitude"][15]
    long = str(y)
    station = bike_yelp_small["station"][15]
    url = "https://api.foursquare.com/v3/places/search?ll%3D"+lat+"%2C"+long+"&radius=500&categories=13065%2C16000%2C10000&sort=DISTANCE&limit=50"
    headers = {
        "Accept": "application/json",
        "Authorization": <API_KEY>
    }
    print(lat,long,station,url)

In [ ]:
#load json object

with open('FS_center.json') as f:
    FS_center = json.load(f)
FS_center = pd.DataFrame(FS_center['results'])
FS_center.head(1)

In [ ]:
foursquare = pd.DataFrame(data['results'])[
        ['fsq_id', 'categories', 'distance', 'geocodes']]

In [ ]:
# Reset the index for your source lat/long file, the code can't run through the index if there are numbers missing.
bike_yelp_small = bike_yelp_small.reset_index()

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

First 'get' for restaurants in Helsinki. I know we were not supposed to do it this way, but I worked from the center of Helsinki and got 
all the results for all restaurants within 3.75kms of that center point.

In [ ]:
import requests
url = "https://api.yelp.com/v3/businesses/search?location=Helsinki&term=restaurants&sort_by=distance&limit=50"
headers = {
    "accept": "application/json",
    "Authorization": "bearer <API_KEY>"
    }
response = requests.get(url, headers=headers)
print(response.text)

Concatenate all the results from all of the calls within the diameter of the center of Helsinki

In [ ]:
yelpconcat = (yelp_restaurant,yelp_restaurant2,yelp_restaurant3,yelp_restaurant4,yelp_restaurant5,yelp_restaurant6,yelp_restaurant7,
             yelp_restaurant8,yelp_restaurant9,yelp_restaurant10,yelp_restaurant11,yelp_restaurant12,yelp_restaurant13,yelp_restaurant14,
             yelp_restaurant15,yelp_restaurant16,yelp_restaurant17,yelp_restaurant18,yelp_restaurant19)

Now, drop all the columns you don't need

In [ ]:
yelp_restaurants = pd.concat(yelpconcat,ignore_index=True)
yelp_restaurants.drop(columns='alias',inplace=True)
yelp_restaurants.drop(columns='image_url',inplace=True)
yelp_restaurants.drop(columns='url',inplace=True)
yelp_restaurants.drop(columns='transactions',inplace=True)
yelp_restaurants.drop(columns='phone',inplace=True)
yelp_restaurants.drop(columns='display_phone',inplace=True)
yelp_restaurants.drop(columns='is_closed',inplace=True)
yelp_restaurants

In [ ]:
#And save it!
yelp_restaurants.to_csv(r'C:\Users\ca0pa\Desktop\LHL\Python\yelp_restaurants.csv', index=False)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
load json object
with open('yelp01.json') as f:
    yelp_restaurant = json.load(f)
yelp_restaurant = pd.DataFrame(yelp_restaurant['businesses'])
yelp_rest = pd.json_normalize(yelp_restaurant['coordinates'])
yelp_restaurant.head()
# bars_2 = pd.DataFrame(bars_1['RangeIndex'])
# yelp_restaurant.explode('coordinates')
# bikes = pd.DataFrame(bikes['network']['stations'])
# bikes

#### I tried to normalize the column 'coordinates' and keep it in the dataframe, but for some reason I can't explain, the
#### panda doesn't want to do that. So, I'll have to normalize the variable and then add it back to the dataframe. ???

In [ ]:
yelp_rest_cood = pd.DataFrame(pd.json_normalize(yelp_restaurants['coordinates']))
yelp_rest_cood.

In [ ]:
test = pd.concat([yelp_restaurants, yelp_rest_cood], axis=1, join="inner")

In [ ]:
yelp_restaurants = pd.DataFrame(test)
yelp_restaurants.head()

Now get rid of what we can't use in the join. 'id', 'coordinates', 'location', 'price'

In [ ]:
yelp_restaurants.drop(columns='id',inplace=True)
yelp_restaurants.drop(columns='coordinates',inplace=True)
yelp_restaurants.drop(columns='location',inplace=True)
yelp_restaurants.drop(columns='price',inplace=True)
yelp_restaurants.drop(columns='categories',inplace=True)

Now, check for duplicate rows that might have been called twice from the api There are None!!

In [ ]:
yelp_restaurants.drop_duplicates()

In [ ]:
#To do what I want to do, I'll have the change the names of the latitude and longitude columns in this data.
yelp_restaurants = pd.DataFrame(yelp_restaurants.rename(columns={"latitude": "lat", "longitude": "lon", "name":"rest_name"}))

In [ ]:
#and, finally, create the variable for the merge
yelp_restaurants['mergevar'] = 1

Put your parsed results into a DataFrame

yelp_restaurants

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Though Yelp was THE WORST to work with as far as their user interface on their website, and the fact that they made you make a 
consumer Yelp account, which is annoying, their API extraction was superior in that their search function readily took keywords instead
of category numbers, and they had an offset variable so you could get more than 50 results from a search point.

In this way, even though there were far fewer results, it may make more sense in some cases to use this data source over FourSquare.

However, the FourSquare data was far more extensive. The search radius provided more than 6x as many results as Yelp.